In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-16 21:50:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.89MB/s    in 0.2s    

2022-02-16 21:50:28 (4.89 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
#Import functions
import pyspark.sql.functions as f
import sys
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [ ]:
from pyspark import SparkFiles
url = "https://amazon-reviews-1.s3.amazonaws.com/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
# DELIVERABLE 2, filter data and create new df total_votes >= 20
top_votes_df = df.filter("total_votes>=20")

In [ ]:
#Filter new df 
greater_than_fifty_df=top_votes_df.filter("(helpful_votes/total_votes)>=0.5")


In [ ]:
#new DataFrame that retrieves all the rows where vine == 'Y'.

#greater_than_fifty_df.printSchema()
vine_paid=greater_than_fifty_df.filter(greater_than_fifty_df.vine == 'Y')


In [ ]:
#unpaid participants
vine_unpaid=greater_than_fifty_df.filter(greater_than_fifty_df.vine == 'N')

In [ ]:
 #Total paid reviews 
total_paid=vine_paid.count()
count_paid=vine_paid.groupBy('star_rating').count()\
.withColumn('Percentage', (col('count')/ total_paid)*100)
total_paid


170

In [ ]:
#Paid reviews with 5 star ratings - count and percentage of total
filtered_df=count_paid.filter(count_paid.star_rating == "5")
filtered_df.select("*",round("Percentage"))\
.withColumnRenamed('round(Percentage, 0)','Rounded Percetage').show()

+-----------+-----+-----------------+-----------------+
|star_rating|count|       Percentage|Rounded Percetage|
+-----------+-----+-----------------+-----------------+
|          5|   65|38.23529411764706|             38.0|
+-----------+-----+-----------------+-----------------+



In [ ]:
 #Total unpaid reviews 
total_unpaid=vine_unpaid.count()
count_unpaid=vine_unpaid.groupBy('star_rating').count()\
.withColumn('Percentage', (col('count')/ total_unpaid)*100)
total_unpaid

37840

In [ ]:
#Unpaid reviews with 5 star ratings - count and percentage of total
filtered_df_unpaid=count_unpaid.filter(count_unpaid.star_rating == "5")
filtered_df_unpaid.select("*",round("Percentage"))\
.withColumnRenamed('round(Percentage, 0)','Rounded Percetage').show()

+-----------+-----+------------------+-----------------+
|star_rating|count|        Percentage|Rounded Percetage|
+-----------+-----+------------------+-----------------+
|          5|20612|54.471458773784356|             54.0|
+-----------+-----+------------------+-----------------+

